# Linear models with CNN features

In [1]:
# Rather than importing everything manually, we'll make things easy
#   and load them all in utils.py, and just import them from there.
%matplotlib inline
import utils; reload(utils)
from utils import *

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: no CUDA-capable device is detected)
Using Theano backend.


Set path and where models should be saved

In [2]:
path = "data/nails/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

Set batch size for how many images to process at a time

In [3]:
#batch_size=64
batch_size=4

We will be doing transfer learning based on a convolutional neural network called VGG. This is a pretty good choice for image recognition tasks as it won ImageNet a few years ago.

In [4]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

First let's try using a linear model from vgg results to our 5 classes.

In [5]:
# Use batch size of 1 since we're just doing preprocessing on the CPU
val_batches = get_batches(path+'valid', shuffle=False, batch_size=1)
batches = get_batches(path+'train', shuffle=False, batch_size=1)

Found 35 images belonging to 5 classes.
Found 157 images belonging to 5 classes.


Compress and save arrays

In [6]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

Join arrays from all batches for validation and training (separately)

In [7]:
val_data = get_data(path+'valid')

Found 35 images belonging to 5 classes.


In [8]:
trn_data = get_data(path+'train')

Found 157 images belonging to 5 classes.


Save array so can be loaded more easily

In [9]:
save_array(model_path+'train_data.bc', trn_data)
save_array(model_path+'valid_data.bc', val_data)

In [10]:
trn_data = load_array(model_path+'train_data.bc')
val_data = load_array(model_path+'valid_data.bc')

In [11]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [12]:
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [43]:
trn_labels

array([[ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  

Make features for linear model

In [13]:
trn_features = model.predict(trn_data, batch_size=batch_size)
val_features = model.predict(val_data, batch_size=batch_size)

In [14]:
save_array(model_path+'train_lastlayer_features.bc', trn_features)
save_array(model_path+'valid_lastlayer_features.bc', val_features)

In [15]:
trn_features = load_array(model_path+'train_lastlayer_features.bc')
val_features = load_array(model_path+'valid_lastlayer_features.bc')

Create a dense model with 1000 inputs (vgg output)

In [16]:
# 1000 inputs, since that's the saved features, and 2 outputs, for dog and cat
lm = Sequential([ Dense(5, activation='softmax', input_shape=(1000,)) ])
lm.compile(optimizer=RMSprop(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
batch_size=4

In [18]:
lm.fit(trn_features, trn_labels, nb_epoch=30, batch_size=batch_size, 
       validation_data=(val_features, val_labels))

Train on 157 samples, validate on 35 samples
Epoch 1/30
157/157 [==============================] - 0s - loss: 1.1634 - acc: 0.5541 - val_loss: 1.0409 - val_acc: 0.4571
Epoch 2/30
157/157 [==============================] - 0s - loss: 0.8406 - acc: 0.6752 - val_loss: 0.8292 - val_acc: 0.6857
Epoch 3/30
157/157 [==============================] - 0s - loss: 0.6756 - acc: 0.7580 - val_loss: 0.7974 - val_acc: 0.6571
Epoch 4/30
157/157 [==============================] - 0s - loss: 0.5767 - acc: 0.7962 - val_loss: 0.7839 - val_acc: 0.7143
Epoch 5/30
157/157 [==============================] - 0s - loss: 0.5066 - acc: 0.8089 - val_loss: 0.7200 - val_acc: 0.7429
Epoch 6/30
157/157 [==============================] - 0s - loss: 0.4446 - acc: 0.8471 - val_loss: 0.7048 - val_acc: 0.7714
Epoch 7/30
157/157 [==============================] - 0s - loss: 0.3948 - acc: 0.8790 - val_loss: 0.7999 - val_acc: 0.7143
Epoch 8/30
157/157 [==============================] - 0s - loss: 0.3498 - acc: 0.8854 - val_lo

In [19]:
lm.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_4 (Dense)                  (None, 5)             5005        dense_input_1[0][0]              
Total params: 5005
____________________________________________________________________________________________________


We have two dense layers stacked. This mathematically is redundant (linear combinations of linear combinations are linear combinations). Let's use one instead.

In [20]:
vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [21]:
model.pop()
for layer in model.layers: layer.trainable=False

In [22]:
model.pop()

**Careful!** Now that we've modified the definition of *model*, be careful not to rerun any code in the previous sections, without first recreating the model from scratch! (Yes, I made that mistake myself, which is why I'm warning you about it now...)

Now we're ready to add our new final layer...

In [23]:
model.add(Dense(5, activation='softmax'))

In [24]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  0           zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [25]:
gen=image.ImageDataGenerator()
batches = gen.flow(trn_data, trn_labels, batch_size=batch_size, shuffle=True)
val_batches = gen.flow(val_data, val_labels, batch_size=batch_size, shuffle=False)

In [26]:
def fit_model(model, batches, val_batches, nb_epoch=1):
    model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=nb_epoch, 
                        validation_data=val_batches, nb_val_samples=val_batches.N)

In [39]:
opt = RMSprop(lr=0.1)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
fit_model(model, batches, val_batches, nb_epoch=8)

Epoch 1/8
157/157 [==============================] - 96s - loss: 7.6424 - acc: 0.5032 - val_loss: 9.2103 - val_acc: 0.4286
Epoch 2/8
157/157 [==============================] - 77s - loss: 7.6997 - acc: 0.5223 - val_loss: 9.2103 - val_acc: 0.4286
Epoch 3/8
157/157 [==============================] - 99s - loss: 7.6997 - acc: 0.5223 - val_loss: 9.2103 - val_acc: 0.4286
Epoch 4/8
157/157 [==============================] - 79s - loss: 7.6997 - acc: 0.5223 - val_loss: 9.2103 - val_acc: 0.4286
Epoch 5/8
 72/157 [============>.................] - ETA: 34s - loss: 6.7159 - acc: 0.5833

KeyboardInterrupt: 

In [29]:
model.save_weights(model_path+'finetune1.h5')

In [30]:
model.load_weights(model_path+'finetune1.h5')

In [31]:
model.evaluate(val_data, val_labels)

KeyboardInterrupt: 

Let's instead train all the dense layers of vgg via backprop

In [32]:
layers = model.layers
first_dense_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense][0]
for layer in layers[first_dense_idx:]: layer.trainable=True

Change learning rate and try fitting again

In [33]:
K.set_value(opt.lr, 0.01)
fit_model(model, batches, val_batches, 3)

Epoch 1/3
  4/157 [..............................] - ETA: 109s - loss: 8.0590 - acc: 0.5000

KeyboardInterrupt: 

In [34]:
model.save_weights(model_path+'finetune2.h5')

KeyboardInterrupt: 

May as well try training some conv layers too, but careful with the learning rate

In [35]:
for layer in layers[12:]: layer.trainable=True
K.set_value(opt.lr, 0.001)

In [36]:
fit_model(model, batches, val_batches, 4)

Epoch 1/4


KeyboardInterrupt: 

In [37]:
model.save_weights(model_path+'finetune3.h5')

In [40]:
model.load_weights(model_path+'finetune3.h5')
model.evaluate_generator(get_batches(path+'valid', gen, False, batch_size*2), val_batches.N)

Found 35 images belonging to 5 classes.


[9.2103402546473951, 0.42857142857142855]